### Import Library

In [1]:
! pip install -r ../requirements.txt

^C


In [2]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

### Extract Data

In [3]:
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
response = requests.get(url)
data = response.json()

earthquakes = []
for feature in data["features"]:
    properties = feature["properties"]
    geometry = feature["geometry"]
    earthquakes.append({
        "id": feature["id"],
        "magnitude": properties["mag"],
        "place": properties["place"],
        "time": properties["time"],
        "longitude": geometry["coordinates"][0],
        "latitude": geometry["coordinates"][1],
        "depth": geometry["coordinates"][2]
    })

df = pd.DataFrame(earthquakes)

In [5]:
df.head()

,id,magnitude,place,time,longitude,latitude,depth
0,nc75181532,0.75,"2 km of The Geysers, CA",1747146985010,-122.765999,38.795666,3.6100
1,tx2025jihc,1.40,"7 km WNW of Westbrook, Texas",1747146512871,-101.091000,32.371000,4.1492
2,ak025646yp7y,2.50,"64 km W of Tyonek, Alaska",1747146354061,-152.322000,61.137600,14.5000
3,ak025646ygj8,1.70,"63 km WNW of Anchor Point, Alaska",1747146281482,-152.890700,59.971500,90.1000
4,ak025646fqre,2.00,"77 km SSE of Perryville, Alaska",1747143429382,-158.764800,55.247800,25.1000


In [6]:
df = df[(df["longitude"].between(95, 141)) & (df["latitude"].between(-11, 6))]

In [7]:
df["depth_category"] = pd.cut(
    df["depth"],
    bins=[0, 70, 300, 700],
    labels=["Shallow", "Intermediate", "Deep"]
)

In [8]:
df["time"] = pd.to_datetime(df["time"], unit="ms")

df["time"] = df["time"].dt.tz_localize("UTC").dt.tz_convert("Asia/Jakarta")

In [9]:
df.head(30)

,id,magnitude,place,time,longitude,latitude,depth,depth_category
64,us7000pyy2,4.4,"173 km W of Gorontalo, Indonesia",2025-05-13 12:41:10.138000+07:00,121.5159,0.6925,100.046,Intermediate
173,us7000pyvj,5.1,"145 km WNW of Tobelo, Indonesia",2025-05-13 00:01:40.278000+07:00,126.7615,2.1083,39.844,Shallow
177,us7000pyus,4.1,"248 km ENE of Lospalos, Timor Leste",2025-05-12 23:23:59.052000+07:00,128.9557,-7.4163,133.637,Intermediate
185,us7000pyul,4.5,"51 km WSW of Sungai Penuh, Indonesia",2025-05-12 22:40:06.773000+07:00,100.9618,-2.2280,69.192,Shallow
312,us7000pyrf,4.9,"50 km WSW of Padangsidempuan, Indonesia",2025-05-12 10:09:38.216000+07:00,98.8950,1.1274,92.219,Intermediate
477,us7000pymn,4.4,"134 km WNW of Ternate, Indonesia",2025-05-11 17:10:38.696000+07:00,126.3581,1.4290,48.677,Shallow
495,us7000pyjn,5.8,"33 km ENE of Blangpidie, Indonesia",2025-05-11 15:57:44.519000+07:00,97.0973,3.8987,94.750,Intermediate
698,us7000pyfm,4.2,"272 km NNE of Lospalos, Timor Leste",2025-05-10 23:24:08.537000+07:00,128.1220,-6.3240,347.013,Deep
728,us7000pyf9,4.5,"178 km W of Abepura, Indonesia",2025-05-10 20:42:55.889000+07:00,139.0368,-2.3871,40.337,Shallow
850,us7000pydg,4.4,"182 km N of Maubara, Timor Leste",2025-05-10 10:34:52.537000+07:00,124.9537,-6.9834,546.512,Deep


In [16]:
load_dotenv()

engine = create_engine(
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

In [25]:
df.to_sql("earthquakes", engine, if_exists="append", index=False)

120

In [26]:
df_all = pd.read_sql("SELECT * FROM earthquakes", engine)
df_all.head(30)

,id,magnitude,place,time,longitude,latitude,depth,depth_category
0,us7000pyy2,4.4,"173 km W of Gorontalo, Indonesia",2025-05-13 05:41:10.138000+00:00,121.5159,0.6925,100.046,Intermediate
1,us7000pyvj,5.1,"145 km WNW of Tobelo, Indonesia",2025-05-12 17:01:40.278000+00:00,126.7615,2.1083,39.844,Shallow
2,us7000pyus,4.1,"248 km ENE of Lospalos, Timor Leste",2025-05-12 16:23:59.052000+00:00,128.9557,-7.4163,133.637,Intermediate
3,us7000pyul,4.5,"51 km WSW of Sungai Penuh, Indonesia",2025-05-12 15:40:06.773000+00:00,100.9618,-2.2280,69.192,Shallow
4,us7000pyrf,4.9,"50 km WSW of Padangsidempuan, Indonesia",2025-05-12 03:09:38.216000+00:00,98.8950,1.1274,92.219,Intermediate
5,us7000pymn,4.4,"134 km WNW of Ternate, Indonesia",2025-05-11 10:10:38.696000+00:00,126.3581,1.4290,48.677,Shallow
6,us7000pyjn,5.8,"33 km ENE of Blangpidie, Indonesia",2025-05-11 08:57:44.519000+00:00,97.0973,3.8987,94.750,Intermediate
7,us7000pyfm,4.2,"272 km NNE of Lospalos, Timor Leste",2025-05-10 16:24:08.537000+00:00,128.1220,-6.3240,347.013,Deep
8,us7000pyf9,4.5,"178 km W of Abepura, Indonesia",2025-05-10 13:42:55.889000+00:00,139.0368,-2.3871,40.337,Shallow
9,us7000pydg,4.4,"182 km N of Maubara, Timor Leste",2025-05-10 03:34:52.537000+00:00,124.9537,-6.9834,546.512,Deep
